In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from point2mn import point2mn
from point2mn import get_main_points
torch.manual_seed(1)

In [6]:
import numpy as np
import pandas as pd

In [7]:
torch.__version__

'1.4.0'

In [8]:
embedding_index = {}

f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()

print('found word vecs: ',len(embedding_index))

found word vecs:  400000


In [5]:
# sorted([s for s in  list(embedding_index.keys()) if s[0].isalpha()])

In [9]:
main_points = pd.read_csv('DebateStar/Meta Data/metadata_appended_main_points.csv') 
main_points.dropna(subset = ['For_Main_Points', 'against_Main_Points'], inplace = True)
main_points = main_points.reset_index(drop = True)
main_points.head()

,id,title,date,for,against,For_Main_Points,against_Main_Points
0,d20191112,Capitalism Is a Blessing,2019-11-12,"['John Mackey', 'Katherine Mangu-Ward']","['Bhaskar Sunkara', 'Richard D. Wolff']",['By promoting market competition and rewardin...,['Capitalism serves the interests of large cor...
1,d20191029,Parenting Is Overrated,2019-10-29,"['Robert Plomin', 'Nancy Segal']","['Paige Harden', 'Ann Pleshette Murphy']","[""We're in the midst of a DNA revolution: Whil...","['While DNA is important, factors like familia..."
2,d20191022,Europe Has Declared War on American Tech Compa...,2019-10-22,"['Roslyn Layton', 'Berin Szóka']","['Marietje Schaake', 'Ramesh Srinivasan']",['European regulators have declared war on Ame...,['Brussels isn’t waging war on Silicon Valley....
3,d20190917,Replace Private Insurance with Medicare for All,2019-09-17,"['Dr. Adam Gaffney', 'Joseph Sanberg']","['Nick Gillespie', 'Sally Pipes']",['The United States government should follow t...,['Individuals should have the freedom to choos...
4,d20190802,The Recent U.S. Policy Towards China Is Produc...,2019-08-02,"['Michael Pillsbury', 'Kori Schake']","['Graham Allison', 'Jake Sullivan']",['The prospect of China becoming an open and l...,['The United States and China are great compet...


In [10]:
results = pd.read_csv('DebateStar/results_data/final_online.csv') 
results = results.loc[results['winner'].apply(lambda x: x != 'undecided')]
results['winner'] = results['winner'].apply(lambda x: 1 if x == 'for' else 0)
id2winner = results[['id', 'winner']].set_index('id').to_dict()['winner']

In [11]:
main_points['label'] = main_points.id.apply(lambda x: id2winner[x] if x in id2winner else np.nan)
main_points = main_points.dropna(subset = ['label']).reset_index(drop = True)
main_points.head()

,id,title,date,for,against,For_Main_Points,against_Main_Points,label
0,d20191112,Capitalism Is a Blessing,2019-11-12,"['John Mackey', 'Katherine Mangu-Ward']","['Bhaskar Sunkara', 'Richard D. Wolff']",['By promoting market competition and rewardin...,['Capitalism serves the interests of large cor...,0.0
1,d20191029,Parenting Is Overrated,2019-10-29,"['Robert Plomin', 'Nancy Segal']","['Paige Harden', 'Ann Pleshette Murphy']","[""We're in the midst of a DNA revolution: Whil...","['While DNA is important, factors like familia...",1.0
2,d20191022,Europe Has Declared War on American Tech Compa...,2019-10-22,"['Roslyn Layton', 'Berin Szóka']","['Marietje Schaake', 'Ramesh Srinivasan']",['European regulators have declared war on Ame...,['Brussels isn’t waging war on Silicon Valley....,0.0
3,d20190917,Replace Private Insurance with Medicare for All,2019-09-17,"['Dr. Adam Gaffney', 'Joseph Sanberg']","['Nick Gillespie', 'Sally Pipes']",['The United States government should follow t...,['Individuals should have the freedom to choos...,0.0
4,d20190802,The Recent U.S. Policy Towards China Is Produc...,2019-08-02,"['Michael Pillsbury', 'Kori Schake']","['Graham Allison', 'Jake Sullivan']",['The prospect of China becoming an open and l...,['The United States and China are great compet...,0.0


In [12]:
inputs = main_points.id.tolist()
labels =  main_points.label.tolist()

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(inputs, labels, test_size = 0.15, random_state = 8)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.15, random_state = 0)

In [15]:
y_train = torch.tensor(y_train)
y_val = torch.tensor(y_val)
y_test = torch.tensor(y_test)


In [16]:
pd.Series(y_test).value_counts(normalize = True)

1.0    0.5
0.0    0.5
dtype: float64

In [17]:
pd.Series(y_val).value_counts(normalize = True)

0.0    0.5
1.0    0.5
dtype: float64

In [18]:
pd.Series(y_train).value_counts(normalize = True)

0.0    0.517647
1.0    0.482353
dtype: float64

In [19]:
print('Training:', len(X_train))
print('Validation:',len(X_val))
print('Testing:',len(X_test))

Training: 85
Validation: 16
Testing: 18


1. Hidden state initialize differently every time running LSTM??

2. init_hidden?

3. Softmax?

4. Use only hidden state, not cell state or output?

5. Results somewhat random?

6. Hidden Dimension?

Fine-tune embedding

Same initial hidden states DONE

Check dropout in evaluation DONE no dropout

Validation set DONE

Check fully connected weights 

Add f.c. layers DOING

Mean + std of several runs



In [20]:
def best_testing(train_acc, val_acc, test_acc, how):
    df = pd.DataFrame({'# Epoch': range(1,epochs + 1), 'training acc': train_acc,\
                         'validation acc': val_acc, 'testing acc': test_acc})
    if how == 'first':
        return df.loc[df['validation acc'] == max(df['validation acc'])].iloc[0]['testing acc']
    elif how == 'last':
        return df.loc[df['validation acc'] == max(df['validation acc'])].iloc[-1]['testing acc']
    elif how == 'best':
        return df.loc[df['validation acc'] == max(df['validation acc'])]['testing acc'].max()
    else:
        print('Invalid Argument for how:', how, "Should be one of 'first', 'last', or 'best'")

In [41]:
def run_lstm_one_sentence(sentence, title, lstm, hidden, embedding_index):
    inputs = point2mn(sentence, title, embedding_index)
    tensor_input = [torch.tensor([x]) for x in inputs]
    tensor_input = torch.cat(tensor_input).view(1, len(tensor_input), -1)
#     hidden = (torch.randn(1, 1, HIDDEN_DIM), torch.randn(1, 1, HIDDEN_DIM)) #???????
    tensor_output, hidden = lstm(tensor_input, hidden)
    # should return hidden state and cell state in 'hidden' instead of output
    return hidden

In [29]:
def run_lstm_on_fid(fid, embedding_index, lstm, hidden, combine_func = [torch.mean, torch.max]):
    """
    Given fid, find main points for the debate and for each sentence, pass corresponding 
    matching vectors to lstm and get hidden state in the end. Gather the hidden states
    in a list and do combine_func.
    -combine_func: the funtion applied to combine lstm outputs from one side elementwisely
    """
    
    title = main_points[main_points['id'] == fid].title.iloc[0]
    for_main_points, against_main_points = get_main_points(fid, main_points)
    for_output_list = []
    against_output_list = []
    
    for sentence in for_main_points:
        hidden_state, cell_state = run_lstm_one_sentence(sentence, title, lstm, hidden, embedding_index)
        for_output_list.append(hidden_state)
        
    for sentence in against_main_points:
        hidden_state, cell_state = run_lstm_one_sentence(sentence, title, lstm, hidden, embedding_index)
        against_output_list.append(hidden_state)
    for_torchs = []
    against_torchs = []
    for combine_f in combine_func:
        if combine_f == torch.mean:
            for_torch = combine_f(torch.stack(for_output_list), dim = 0, keepdim = True)#[0]
            against_torch = combine_f(torch.stack(against_output_list), dim = 0, keepdim = True)#[0]
        else:
            for_torch = combine_f(torch.stack(for_output_list), dim = 0, keepdim = True)[0]
            against_torch = combine_f(torch.stack(against_output_list), dim = 0, keepdim = True)[0]
        for_torchs.append(for_torch)
        against_torchs.append(against_torch)
    if len(for_torchs) == 2:
        t1 = torch.cat((for_torchs[0], for_torchs[1]), dim = 2)
        t2 = torch.cat((against_torchs[0], against_torchs[1]), dim = 2)
        return torch.cat((t1, t2), dim = 2)
    return torch.cat((for_torchs[0], against_torchs[0]), dim = 2)

In [30]:
class LSTMNet(nn.Module):
    def __init__(self, output_size_1, output_size_2, n_layers, embedding_dim, hidden_dim, combine_funcs, hidden, drop_prob=0.5):
        super(LSTMNet, self).__init__()
        self.output_size_1 = output_size_1
        self.output_size_2 = output_size_2
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.combine_funcs = combine_funcs
        self.hidden = hidden

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc1 = nn.Linear(hidden_dim * 2 * len(self.combine_funcs), self.output_size_1)
        self.fc2 = nn.Linear(output_size_1, self.output_size_2)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, fids):
        batch_size = len(fids)
        lstm_out = torch.stack([run_lstm_on_fid(fid, embedding_index, self.lstm, self.hidden, \
                                                combine_func = self.combine_funcs) for fid in fids])
    
        lstm_out = lstm_out.contiguous().view(-1, len(self.combine_funcs) * 2 * self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out#, hidden
    
#     def init_hidden(self, batch_size):
#         weight = next(self.parameters()).data
#         hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
#                       weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
#         return hidden

In [40]:
lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True)
hidden = (torch.randn(1, 1, HIDDEN_DIM), torch.randn(1, 1, HIDDEN_DIM))
run_lstm_one_sentence('It is just a test test test', 'this is a debate title', lstm, hidden, embedding_index)

(tensor([[[-2.2285e-01, -2.9363e-01,  4.1848e-02,  2.4278e-01,  9.8378e-02,
           -5.4527e-02, -1.9896e-02,  3.5270e-01, -8.9105e-02, -1.5113e-01,
            1.8225e-01,  2.4688e-01, -1.0039e-01, -3.1386e-01,  9.1738e-02,
            6.7223e-02,  4.2452e-01,  2.8536e-01, -2.3161e-01,  2.1571e-02,
            1.2818e-01, -4.5801e-01, -2.5654e-01, -2.3639e-01, -6.6850e-02,
            1.7433e-01, -1.2748e-01, -4.6969e-02, -1.7209e-02,  4.4396e-03,
           -3.2286e-01, -2.7042e-01, -4.6739e-02, -7.8938e-02, -3.2179e-01,
            6.0513e-01, -2.4542e-01,  1.7760e-02, -1.6119e-01, -2.0208e-01,
            6.9686e-02,  2.1417e-01,  1.8965e-01, -2.5910e-01, -1.4694e-01,
            3.3707e-01, -1.6223e-01, -1.4860e-01,  6.2354e-02, -2.3028e-01],
          [-4.1652e-02, -2.9644e-01, -7.1542e-03,  2.6875e-01,  1.3184e-02,
           -1.1651e-01,  3.2514e-02,  4.7123e-02,  5.7037e-02, -1.0140e-01,
            3.5391e-01,  2.6118e-01, -1.1132e-01, -1.5567e-01,  1.9504e-01,
           

In [31]:
EMBEDDING_DIM = 200
HIDDEN_DIM = 50
OUTPUT_SIZE_1 = 20
COMBINE_FUNCS =[torch.mean, torch.max]
OUTPUT_SIZE_2 = 1
N_LAYERS = 1
DROPOUT_PROB = 0.5
HIDDEN_INITIAL = (torch.randn(1, 1, HIDDEN_DIM), torch.randn(1, 1, HIDDEN_DIM))

# all should use the same one lstm layer
# lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM)

In [32]:
model = LSTMNet(OUTPUT_SIZE_1, OUTPUT_SIZE_2, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, \
                COMBINE_FUNCS, HIDDEN_INITIAL,DROPOUT_PROB)

In [33]:
lr = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [34]:
epochs = 20
counter = 0
# print_every = 1000
# clip = 5
# valid_loss_min = np.Inf
train_acc = []
train_acc2 = []

val_acc = []
test_acc = []
model.train()
for i in range(epochs):
#     h = model.init_hidden(batch_size)
    
#     for inputs, labels in train_loader:
#         counter += 1
#         h = tuple([e.data for e in h])
#         inputs, labels = inputs.to(device), labels.to(device)
    model.train()
    model.zero_grad()
    output = model(X_train)
    loss = criterion(output.squeeze(), y_train.float())
    print(loss.item())
    loss.backward()
#     nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    model.eval()
    outs = model(X_train)
    train_acc.append(torch.sum((outs > 0.5) == y_train).item() / len(y_train))

    outs = model(X_val)
    val_acc.append(torch.sum((outs > 0.5) == y_val).item() / len(y_val))
    
    outs = model(X_test)
    test_acc.append(torch.sum((outs > 0.5) == y_test).item() / len(y_test))
    print(train_acc, val_acc)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


KeyboardInterrupt: 

## HIDDEN_DIM = 50, output_size_1 = 10, 2 fcs, [torch.mean, torch.max]¶

In [31]:
EMBEDDING_DIM = 200
HIDDEN_DIM = 50
OUTPUT_SIZE_1 = 10
COMBINE_FUNCS =[torch.mean, torch.max]
OUTPUT_SIZE_2 = 1
N_LAYERS = 1
DROPOUT_PROB = 0.5
HIDDEN_INITIAL = (torch.randn(1, 1, HIDDEN_DIM), torch.randn(1, 1, HIDDEN_DIM))

scores_first = []
scores_last = []
scores_best = []
scores_ideal = []
for i in range(5):
    model = LSTMNet(OUTPUT_SIZE_1, OUTPUT_SIZE_2, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, \
                COMBINE_FUNCS, HIDDEN_INITIAL,DROPOUT_PROB)
    lr = 0.01
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    print(i)
    epochs = 100
    train_acc = []
    val_acc = []
    test_acc = []
    model.train()
    for j in range(epochs):
        model.train()
        model.zero_grad()
        output = model(X_train)
        loss = criterion(output.squeeze(), y_train.float())
        loss.backward()
        print(j, loss.item())
    #     nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        model.eval()
        outs = model(X_train)
        train_acc.append(torch.sum((outs > 0.5) == y_train).item() / len(y_train))

        outs = model(X_val)
        val_acc.append(torch.sum((outs > 0.5) == y_val).item() / len(y_val))

        outs = model(X_test)
        test_acc.append(torch.sum((outs > 0.5) == y_test).item() / len(y_test))
    scores_first.append(best_testing(train_acc, val_acc, test_acc, 'first'))
    scores_last.append(best_testing(train_acc, val_acc, test_acc, 'last'))
    scores_best.append(best_testing(train_acc, val_acc, test_acc, 'best'))
    scores_ideal.append(max(test_acc))

0
0 0.717143714427948
1 0.706023097038269
2 0.6767569780349731
3 0.6614444255828857
4 0.6306944489479065
5 0.6063251495361328
6 0.5509687066078186
7 0.5286946296691895
8 0.4603004455566406
9 0.375855416059494
10 0.36038756370544434
11 0.4040611684322357
12 0.24962647259235382
13 0.2764521837234497
14 0.15850308537483215
15 0.22365713119506836
16 0.11958719044923782
17 0.14815734326839447
18 0.05993449687957764
19 0.06801258772611618
20 0.06226031482219696
21 0.046341605484485626
22 0.028158899396657944
23 0.07814288139343262
24 0.03282732516527176
25 0.015446184203028679
26 0.021061528474092484
27 0.009942620992660522
28 0.008610766381025314
29 0.002227829536423087
30 0.003399194683879614
31 0.008938322775065899
32 0.007289078552275896
33 0.002762146992608905
34 0.0026610377244651318
35 0.00047819383325986564
36 0.0061164433136582375
37 0.0010530544677749276
38 0.0003746900474652648
39 0.0004252711951266974
40 0.0010435888543725014
41 0.0010059747146442533
42 0.0002494642394594848
43 0

41 0.004919049795717001
42 0.02226395532488823
43 0.004142886493355036
44 0.00392482103779912
45 0.0030320847872644663
46 0.005289600230753422
47 0.0010685318848118186
48 0.0012341232504695654
49 0.0020123557187616825
50 0.0048845550045371056
51 0.005941338371485472
52 0.004332779906690121
53 0.0008416922646574676
54 0.0006986981024965644
55 0.000515699153766036
56 0.0007105037220753729
57 0.0024445513263344765
58 0.0009256525663658977
59 0.0001923310774145648
60 0.001432589953765273
61 0.0003371386555954814
62 0.0010056642349809408
63 0.0020551939960569143
64 0.0002942959254141897
65 0.00039617024594917893
66 0.0002866090217139572
67 0.00010203697456745431
68 8.452918700641021e-05
69 0.000345304433722049
70 0.0001613888452993706
71 0.00010912789002759382
72 0.0001877938339021057
73 0.00010561483213678002
74 0.00012802916171494871
75 0.00011460151290521026
76 0.0002159714640583843
77 0.0005787283880636096
78 0.0001610374019946903
79 0.00046687599387951195
80 0.00021405829465948045
81 0

In [32]:
print(scores_first)
print(scores_last)
print(scores_best)

[0.6666666666666666, 0.5555555555555556, 0.6111111111111112, 0.6111111111111112, 0.7222222222222222]
[0.6666666666666666, 0.6111111111111112, 0.4444444444444444, 0.6111111111111112, 0.7222222222222222]
[0.6666666666666666, 0.6111111111111112, 0.6111111111111112, 0.6666666666666666, 0.7222222222222222]


In [33]:
scores_ideal

[0.7222222222222222,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.7222222222222222]

In [59]:
print(np.mean([0.6666666666666666, 0.5555555555555556, 0.6111111111111112, 0.6111111111111112, 0.7222222222222222]))
print(np.mean([0.6666666666666666, 0.6111111111111112, 0.4444444444444444, 0.6111111111111112, 0.7222222222222222]))
print(np.mean([0.6666666666666666, 0.6111111111111112, 0.6111111111111112, 0.6666666666666666, 0.7222222222222222]))


0.6333333333333334
0.611111111111111
0.6555555555555556


In [19]:
def experiments(OUTPUT_SIZE_1, OUTPUT_SIZE_2, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, \
                COMBINE_FUNCS, HIDDEN_INITIAL,DROPOUT_PROB, epochs, iterations):
    scores_first = []
    scores_last = []
    scores_best = []
    scores_ideal = []
    for i in range(iterations):
        model = LSTMNet(OUTPUT_SIZE_1, OUTPUT_SIZE_2, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, \
                    COMBINE_FUNCS, HIDDEN_INITIAL,DROPOUT_PROB)
        lr = 0.01
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        print(i)
        epochs = 100
        train_acc = []
        val_acc = []
        test_acc = []
        model.train()
        for j in range(epochs):
            model.train()
            model.zero_grad()
            output = model(X_train)
            loss = criterion(output.squeeze(), y_train.float())
            loss.backward()
            print(j, loss.item())
        #     nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            model.eval()
            outs = model(X_train)
            train_acc.append(torch.sum((outs > 0.5) == y_train).item() / len(y_train))

            outs = model(X_val)
            val_acc.append(torch.sum((outs > 0.5) == y_val).item() / len(y_val))

            outs = model(X_test)
            test_acc.append(torch.sum((outs > 0.5) == y_test).item() / len(y_test))
        scores_first.append(best_testing(train_acc, val_acc, test_acc, 'first'))
        scores_last.append(best_testing(train_acc, val_acc, test_acc, 'last'))
        scores_best.append(best_testing(train_acc, val_acc, test_acc, 'best'))
        scores_ideal.append(max(test_acc))
    return {'scores_first': scores_first, 'scores_last':scores_last, 'scores_best': scores_best,\
            'scores_ideal': scores_ideal}

In [26]:
EMBEDDING_DIM = 200
HIDDEN_DIM = 50
OUTPUT_SIZE_1 = 20
COMBINE_FUNCS =[torch.mean, torch.max]
OUTPUT_SIZE_2 = 1
N_LAYERS = 1
DROPOUT_PROB = 0.5
HIDDEN_INITIAL = (torch.randn(1, 1, HIDDEN_DIM), torch.randn(1, 1, HIDDEN_DIM))

scores_first = []
scores_last = []
scores_best = []
scores_ideal = []
for i in range(5):
    model = LSTMNet(OUTPUT_SIZE_1, OUTPUT_SIZE_2, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, \
                COMBINE_FUNCS, HIDDEN_INITIAL, DROPOUT_PROB)
    lr = 0.01
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    print(i)
    epochs = 100
    train_acc = []
    val_acc = []
    test_acc = []
    model.train()
    for j in range(epochs):
        model.train()
        model.zero_grad()
        output = model(X_train)
        loss = criterion(output.squeeze(), y_train.float())
        loss.backward()
        print(j, loss.item())
    #     nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        model.eval()
        outs = model(X_train)
        train_acc.append(torch.sum((outs > 0.5) == y_train).item() / len(y_train))

        outs = model(X_val)
        val_acc.append(torch.sum((outs > 0.5) == y_val).item() / len(y_val))

        outs = model(X_test)
        test_acc.append(torch.sum((outs > 0.5) == y_test).item() / len(y_test))
    scores_first.append(best_testing(train_acc, val_acc, test_acc, 'first'))
    scores_last.append(best_testing(train_acc, val_acc, test_acc, 'last'))
    scores_best.append(best_testing(train_acc, val_acc, test_acc, 'best'))
    scores_ideal.append(max(test_acc))
print(scores_first)
print(scores_last)
print(scores_best)
print(scores_ideal)

0
0 0.692461371421814
1 0.6992563009262085
2 0.7053745985031128
3 0.6433932185173035
4 0.6455641388893127
5 0.5831127166748047
6 0.5695322155952454
7 0.5123856663703918
8 0.4302937984466553
9 0.46260249614715576
10 0.5037082433700562
11 0.28780397772789
12 0.3343343436717987
13 0.23296186327934265
14 0.1974794715642929
15 0.18263038992881775
16 0.12879228591918945
17 0.10432902723550797
18 0.09088485687971115
19 0.03890206292271614
20 0.08015084266662598
21 0.03352430462837219
22 0.011639142408967018
23 0.009348778985440731
24 0.013669260777533054
25 0.011623789556324482
26 0.0026759812608361244
27 0.0038343940395861864
28 0.005150455050170422
29 0.002095646457746625
30 0.0017128227045759559
31 0.0017323895590379834
32 0.0004983210819773376
33 0.002153074834495783
34 0.000864803500007838
35 0.00015211457503028214
36 0.00020961319387424737
37 0.0010445406660437584
38 0.0002412034955341369
39 7.070325955282897e-05
40 0.00010290680802427232
41 0.003673143684864044
42 0.0007030570995993912

40 0.00439026253297925
41 0.0017909924499690533
42 0.0002958676777780056
43 0.003625099081546068
44 0.0004250875790603459
45 0.0005669035599566996
46 0.0003442463930696249
47 0.000421640754211694
48 0.0014660283923149109
49 5.894234709558077e-05
50 0.00016095150203909725
51 0.0001521232770755887
52 0.00036158348666504025
53 0.0001563687255838886
54 6.47132474114187e-05
55 0.00015076583076734096
56 7.052700675558299e-05
57 0.00018036054098047316
58 0.0003520035825204104
59 0.000538146123290062
60 0.0010035440791398287
61 0.00024292523448821157
62 7.560364610981196e-05
63 5.339087874745019e-05
64 0.00021208969701547176
65 0.0007186196162365377
66 0.00020314678840804845
67 6.187775579746813e-05
68 4.7353758418466896e-05
69 0.000153830973431468
70 0.00016228716413024813
71 0.0005045332363806665
72 0.00017113178910221905
73 0.0006377322133630514
74 7.073706365190446e-05
75 7.397789158858359e-05
76 0.00027394830249249935
77 0.00017807501717470586
78 0.0007404926000162959
79 0.000123002042528

In [42]:
iterations = 5
df = pd.DataFrame({'# Iteration': list(range(1,iterations + 1)) + ['mean'], \
                   'scores_first': scores_first + [np.mean(scores_first)],\
                         'scores_last': scores_last + [np.mean(scores_last)], \
                   'scores_best': scores_best + [np.mean(scores_best)], \
                   'scores_ideal': scores_ideal + [np.mean(scores_ideal)]})
df

,# Iteration,scores_first,scores_last,scores_best,scores_ideal
0,1,0.555556,0.555556,0.555556,0.666667
1,2,0.611111,0.500000,0.611111,0.666667
2,3,0.555556,0.666667,0.666667,0.833333
3,4,0.611111,0.666667,0.666667,0.722222
4,5,0.666667,0.666667,0.666667,0.722222
5,mean,0.600000,0.611111,0.633333,0.722222


## HIDDEN_DIM = 30, output_size_1 = 10, 2 fcs, [torch.mean, torch.max]

In [31]:
EMBEDDING_DIM = 200
HIDDEN_DIM = 30
OUTPUT_SIZE_1 = 10
COMBINE_FUNCS =[torch.mean, torch.max]
OUTPUT_SIZE_2 = 1
N_LAYERS = 1
DROPOUT_PROB = 0.5
HIDDEN_INITIAL = (torch.randn(1, 1, HIDDEN_DIM), torch.randn(1, 1, HIDDEN_DIM))
results = experiments(OUTPUT_SIZE_1, OUTPUT_SIZE_2, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, \
                COMBINE_FUNCS, HIDDEN_INITIAL,DROPOUT_PROB, epochs, iterations)

0
0 0.7209392786026001
1 0.6991881132125854
2 0.6680363416671753
3 0.6521626710891724
4 0.6394084095954895
5 0.6030824780464172
6 0.5655356049537659
7 0.5466874241828918
8 0.45585665106773376
9 0.43434277176856995
10 0.3985781967639923
11 0.3104100525379181
12 0.2596261501312256
13 0.2660207450389862
14 0.16194190084934235
15 0.18888992071151733
16 0.10979588329792023
17 0.15166625380516052
18 0.08165708929300308
19 0.07200481742620468
20 0.017080184072256088
21 0.04558799788355827
22 0.02976216934621334
23 0.03545723855495453
24 0.006710705813020468
25 0.021947519853711128
26 0.014621886424720287
27 0.016545668244361877
28 0.010586890392005444
29 0.004534859210252762
30 0.007739148568361998
31 0.01846708543598652
32 0.010698042809963226
33 0.003819498233497143
34 0.0007363728946074843
35 0.010628625750541687
36 0.0018541236640885472
37 0.0008499554242007434
38 0.0007881139754317701
39 0.0010979649377986789
40 0.004891460761427879
41 0.000523023831192404
42 0.01968265138566494
43 0.000

44 0.0006129975081421435
45 0.0007232955540530384
46 0.0037225179839879274
47 0.00011051290493924171
48 0.000278386112768203
49 0.0019310573115944862
50 0.0006982499035075307
51 0.0006398421246558428
52 0.00015833514044061303
53 6.532363477163017e-05
54 0.00133828679099679
55 6.685681728413329e-05
56 3.307902443339117e-05
57 7.685791206313297e-05
58 1.5120596799533814e-05
59 6.839688285253942e-05
60 7.24118945072405e-05
61 5.897176743019372e-05
62 0.0003897732531186193
63 4.200945113552734e-05
64 8.500077819917351e-05
65 4.8548521590419114e-05
66 1.3505799870472401e-05
67 8.93446704139933e-05
68 0.0001344435877399519
69 2.7077181584900245e-05
70 3.336006557219662e-05
71 1.8740067389444448e-05
72 3.682380338432267e-05
73 3.250363442930393e-05
74 0.00010588821896817535
75 1.736564445309341e-05
76 2.9610966521431692e-05
77 0.00012443590094335377
78 0.00015381211414933205
79 4.195485234959051e-05
80 0.00017757950990926474
81 0.00029330619145184755
82 0.00020530539040919393
83 1.70738221640

In [32]:
df = pd.DataFrame({'# Iteration': list(range(1,iterations + 1)) + ['mean', 'std'], \
    'scores_first': results['scores_first'] + [np.mean(results['scores_first']), np.std(results['scores_first'])],\
     'scores_last': results['scores_last'] + [np.mean(results['scores_last']), np.std(results['scores_last'])], \
     'scores_best': results['scores_best'] + [np.mean(results['scores_best']), np.std(results['scores_best'])], \
    'scores_ideal': results['scores_ideal'] + [np.mean(results['scores_ideal']), np.std(results['scores_ideal'])]})
df

,# Iteration,scores_first,scores_last,scores_best,scores_ideal
0,1,0.611111,0.611111,0.611111,0.722222
1,2,0.611111,0.722222,0.722222,0.777778
2,3,0.611111,0.611111,0.611111,0.777778
3,4,0.722222,0.555556,0.722222,0.722222
4,5,0.666667,0.500000,0.666667,0.722222
5,mean,0.644444,0.600000,0.666667,0.744444
6,std,0.044444,0.073703,0.049690,0.027217


## HIDDEN_DIM = 50, 1 fc, [torch.mean]

In [47]:
EMBEDDING_DIM = 200
HIDDEN_DIM = 50
OUTPUT_SIZE_1 = 1
COMBINE_FUNCS =[torch.mean]
OUTPUT_SIZE_2 = 1
N_LAYERS = 1
DROPOUT_PROB = 0.5
HIDDEN_INITIAL = (torch.randn(1, 1, HIDDEN_DIM), torch.randn(1, 1, HIDDEN_DIM))
results = experiments(OUTPUT_SIZE_1, OUTPUT_SIZE_2, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, \
                COMBINE_FUNCS, HIDDEN_INITIAL,DROPOUT_PROB, epochs, iterations)

0
0 0.6917464137077332
1 0.6984074711799622
2 0.6921167969703674
3 0.6424497961997986
4 0.6421229839324951
5 0.6335413455963135
6 0.6136420965194702
7 0.607666015625
8 0.5730096697807312
9 0.5423061847686768
10 0.48941242694854736
11 0.4433946907520294
12 0.490826815366745
13 0.4872857928276062
14 0.358762264251709
15 0.42498236894607544
16 0.3426423966884613
17 0.3412606120109558
18 0.32328230142593384
19 0.28157785534858704
20 0.26916033029556274
21 0.22325706481933594
22 0.181879922747612
23 0.17620690166950226
24 0.11562053114175797
25 0.12123742699623108
26 0.21593032777309418
27 0.24342581629753113
28 0.10030799359083176
29 0.22849006950855255
30 0.10007015615701675
31 0.09306840598583221
32 0.1320171058177948
33 0.07486504316329956
34 0.07699944078922272
35 0.11206527054309845
36 0.048772312700748444
37 0.04315643385052681
38 0.056020401418209076
39 0.044992025941610336
40 0.025481002405285835
41 0.019321560859680176
42 0.034956835210323334
43 0.03182835876941681
44 0.0139805162

47 0.0019700515549629927
48 0.003989158198237419
49 0.0020789902191609144
50 0.0012155472068116069
51 0.0016836341237649322
52 0.001320175244472921
53 0.0011562724830582738
54 0.0011598257115110755
55 0.0012710748706012964
56 0.0010385116329416633
57 0.0015247728442773223
58 0.000772516883444041
59 0.000760897877626121
60 0.0008881559479050338
61 0.0008014923660084605
62 0.00047970449668355286
63 0.00033587307552807033
64 0.00035551132168620825
65 0.0006341732223518193
66 0.0005060440744273365
67 0.0003190208226442337
68 0.00040816402179189026
69 0.00042667187517508864
70 0.00036553724203258753
71 0.00029603857547044754
72 0.0004606864822562784
73 0.0003319754614494741
74 0.0003418217529542744
75 0.00022458832245320082
76 0.0006119210738688707
77 0.0002907472662627697
78 0.00025715981610119343
79 0.0003364496515132487
80 0.00024925655452534556
81 0.00018848941545002162
82 0.00021933167590759695
83 0.00022413836268242449
84 0.00015108825755305588
85 0.00015611988783348352
86 0.000197737

In [45]:
df = pd.DataFrame({'# Iteration': list(range(1,iterations + 1)) + ['mean'], \
                   'scores_first': results['scores_first'] + [np.mean(results['scores_first'])],\
                         'scores_last': results['scores_last'] + [np.mean(results['scores_last'])], \
                   'scores_best': results['scores_best'] + [np.mean(results['scores_best'])], \
                   'scores_ideal': results['scores_ideal'] + [np.mean(results['scores_ideal'])]})
df

,# Iteration,scores_first,scores_last,scores_best,scores_ideal
0,1,0.555556,0.555556,0.555556,0.722222
1,2,0.555556,0.555556,0.555556,0.722222
2,3,0.666667,0.611111,0.666667,0.722222
3,4,0.500000,0.388889,0.500000,0.666667
4,5,0.611111,0.666667,0.666667,0.722222
5,mean,0.577778,0.555556,0.588889,0.711111


## HIDDEN_DIM = 50, output_size_1 = 20, 2 fcs, [torch.mean, torch.max]

In [28]:
EMBEDDING_DIM = 200
HIDDEN_DIM = 50
OUTPUT_SIZE_1 = 20
COMBINE_FUNCS =[torch.mean, torch.max]
OUTPUT_SIZE_2 = 1
N_LAYERS = 1
DROPOUT_PROB = 0.5
epochs = 100
iterations = 5
HIDDEN_INITIAL = (torch.randn(1, 1, HIDDEN_DIM), torch.randn(1, 1, HIDDEN_DIM))
results = experiments(OUTPUT_SIZE_1, OUTPUT_SIZE_2, N_LAYERS, EMBEDDING_DIM, HIDDEN_DIM, \
                COMBINE_FUNCS, HIDDEN_INITIAL,DROPOUT_PROB, epochs, iterations)

0
0 0.7045242786407471
1 0.7154113054275513
2 0.7108703851699829
3 0.665443480014801
4 0.6484684348106384
5 0.6370669007301331
6 0.6088412404060364
7 0.5580205321311951
8 0.5120642781257629
9 0.4456539750099182
10 0.34897199273109436
11 0.39847105741500854
12 0.7809348702430725
13 0.21073146164417267
14 0.4467383027076721
15 0.365533709526062
16 0.22151866555213928
17 0.24302572011947632
18 0.2557424306869507
19 0.24726556241512299
20 0.17857974767684937
21 0.12845654785633087
22 0.1254992038011551
23 0.12065384536981583
24 0.09062220901250839
25 0.06133587658405304
26 0.03803260624408722
27 0.02782885544002056
28 0.017187386751174927
29 0.014478539116680622
30 0.007680822163820267
31 0.024162663146853447
32 0.007301937323063612
33 0.006052274722605944
34 0.02777852676808834
35 0.01605812832713127
36 0.0021910264622420073
37 0.0014310150872915983
38 0.0019032306736335158
39 0.0016822745092213154
40 0.000491370155941695
41 0.0005104023148305714
42 0.0020333316642791033
43 0.001489729620

43 0.000628416077233851
44 0.0001810549001675099
45 0.0017202625749632716
46 0.000623039435595274
47 0.0003799257392529398
48 0.0028795134276151657
49 0.00437328452244401
50 0.00025651275063864887
51 0.00040713942144066095
52 0.00029229180654510856
53 0.0017371461726725101
54 0.001023960066959262
55 0.0003258329234085977
56 0.0023298312444239855
57 0.00021790560276713222
58 0.000845497299451381
59 0.00015100630116648972
60 9.433188097318634e-05
61 0.00022908720711711794
62 0.0003221928491257131
63 0.00028459238819777966
64 0.003101208945736289
65 0.0029959450475871563
66 0.0035998562816530466
67 0.0005392826860770583
68 0.00011578160774661228
69 0.0016244299476966262
70 0.0005260410835035145
71 0.0006465267506428063
72 0.003769532311707735
73 0.0023318829480558634
74 0.000972720212303102
75 0.00019358292047400028
76 0.0002549319760873914
77 4.130154411541298e-05
78 0.00035655396641232073
79 7.363262557191774e-05
80 0.0015475312247872353
81 0.00031678020604886115
82 0.000665205006953328

In [30]:
df = pd.DataFrame({'# Iteration': list(range(1,iterations + 1)) + ['mean'], \
                   'scores_first': results['scores_first'] + [np.mean(results['scores_first'])],\
                         'scores_last': results['scores_last'] + [np.mean(results['scores_last'])], \
                   'scores_best': results['scores_best'] + [np.mean(results['scores_best'])], \
                   'scores_ideal': results['scores_ideal'] + [np.mean(results['scores_ideal'])]})
df

,# Iteration,scores_first,scores_last,scores_best,scores_ideal
0,1,0.666667,0.666667,0.666667,0.722222
1,2,0.611111,0.611111,0.611111,0.777778
2,3,0.666667,0.611111,0.666667,0.722222
3,4,0.611111,0.666667,0.666667,0.777778
4,5,0.611111,0.611111,0.611111,0.777778
5,mean,0.633333,0.633333,0.644444,0.755556


# nvidia-smi
